# 安装各种包

* #%pip install soundfile
* %pip install openai-whisper
* %pip install setuptools-rust
* %pip install pyaudio
* %pip install SpeechRecognition
* %pip install ipywidgets
'''

* [openai whisper offline model](https://github.com/openai/whisper)

* ↓修改下面的 openai key，用于从gpt自动回答问题

In [1]:
import openai
openai.api_key = '改成你自己的key'

In [2]:
import whisper
from io import BytesIO
import numpy as np
import soundfile as sf
from numba import jit

model = whisper.load_model("small")

/Users/xiayun/miniconda3/envs/py311/lib/python3.11/site-packages/whisper/timing.py:57: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit


In [3]:
import json
import threading
import speech_recognition as sr
import requests

# 创建语音识别器对象
recog = sr.Recognizer()
mic = sr.Microphone()

recording = False
stopper = None

from IPython.display import display
import ipywidgets as widgets

record_button = widgets.Button(
    description='Record',
    disabled=False,
    button_style='success',
    tooltip='Record',
    icon='microphone'
)

stop_button = widgets.Button(
    description='Stop',
    disabled=False,
    button_style='warning',
    tooltip='Stop',
    icon='stop'
)

clear_button = widgets.Button(
    description='Clear',
    disabled=False,
    button_style='info',
    tooltip='Clear',
    icon='trash'
)

output = widgets.Output()

def start_recording(data):
    with output:
        global recording
        recording = True
        listen()

def stop_recording(data):
    with output:
        global recording
        recording = False
        stopper(wait_for_stop=False)
        append_text("录音已停止")
    
def clear_out(data):
    output.clear_output()
    
def append_text(text):
    output.append_display_data(text)
        
def listen():
    # 降低噪音
    #with mic:
    #    recog.adjust_for_ambient_noise(mic)

    # 开始在后台录音
    append_text("请开始说话...")
    global stopper
    stopper = recog.listen_in_background(mic, callback, phrase_time_limit=15)
    
def callback(recognizer, audio):
    append_text('已获取语音')
    t_transcribe = threading.Thread(target=transcribe, args=(audio,))
    t_transcribe.start()

record_button.on_click(start_recording)
stop_button.on_click(stop_recording)
clear_button.on_click(clear_out)

display(record_button, stop_button, clear_button, output)

Button(button_style='success', description='Record', icon='microphone', style=ButtonStyle(), tooltip='Record')

Button(button_style='warning', description='Stop', icon='stop', style=ButtonStyle(), tooltip='Stop')

Button(button_style='info', description='Clear', icon='trash', style=ButtonStyle(), tooltip='Clear')

Output()

In [4]:
def voice_to_text(audio_data):
    # 16 kHz https://github.com/openai/whisper/blob/28769fcfe50755a817ab922a7bc83483159600a9/whisper/audio.py#L98-L99
    wav_bytes = audio_data.get_wav_data(convert_rate=16000)
    wav_stream = BytesIO(wav_bytes)
    audio_array, sampling_rate = sf.read(wav_stream)
    audio_array = audio_array.astype(np.float32)

    prompt = '这是一场java面试，可能出现：开源、数组、链表、指针、hashmap、函数'
    result = model.transcribe(audio_array, language='chinese', verbose=True, initial_prompt=prompt)
    return "success", result["text"]

def text_to_answer(text):    
    MODEL = "gpt-3.5-turbo"
    user_text = text + "。以上这段文本中，是否有java工程师相关的面试问题？如果有，请回答这个问题，回答需要结构化、精炼，控制在100字以内。格式：1. 问题：xxx；2.答案：xxx。如果没有，请返回'不需回答'。"
    response = openai.ChatCompletion.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": "我是一个面试官，你是求职者。现在正在进行一场面试。"},
            {"role": "user", "content": user_text},
        ],
        temperature=0,
    )
    
    return True, response.choices[0].message.content

def transcribe(audio):
    try:
        append_text("识别中...")
        status, text = voice_to_text(audio)
        append_text("识别成功：" + text if status else "识别失败")
        if not status:
            return

        # 回答问题
        append_text("请求回答...")
        status, answer = text_to_answer(text)
        append_text("参考回答：" + answer if status else "请求回答失败")
    except Exception as ex:
        append_text("转换失败：" + ex)